### Classifying Normal and Anomalous HTTP Requests Using Machine Learning

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Load the CSV file
file_path = 'combined_parsed_requests.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataframe
df.head()


,Method,Full URL,URL Path,Query Params,Query Params Length,Number of Query Params,Body Params,Content-Length,Content-Type,User-Agent,...,Accept,Accept-Encoding,Accept-Charset,Accept-Language,Pragma,Connection,Body Length,URL Length,Special Characters Count in URL,Special Characters Count in Query Params
0,GET,http://localhost:8080/tienda1/imagenes/2.gif,/tienda1/imagenes/2.gif,NaN,0,0,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,close,0,44,3,0
1,GET,http://localhost:8080/tienda1/miembros/editar....,/tienda1/miembros/editar.jsp,modo=registro; login=thege; password=abstruso;...,249,13,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,close,0,299,3,28
2,GET,http://localhost:8080/tienda1/publico/carrito.jsp,/tienda1/publico/carrito.jsp,NaN,0,0,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,close,0,49,3,0
3,POST,http://localhost:8080/tienda1/publico/entrar.jsp,/tienda1/publico/entrar.jsp,NaN,0,0,NaN,33.0,application/x-www-form-urlencoded,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,close,0,48,3,0
4,GET,http://localhost:8080/tienda1/miembros/salir.jsp,/tienda1/miembros/salir.jsp,NaN,0,0,NaN,NaN,NaN,Mozilla/5.0 (compatible; Konqueror/3.5; Linux)...,...,"text/xml,application/xml,application/xhtml+xml...","x-gzip, x-deflate, gzip, deflate","utf-8, utf-8;q=0.5, *;q=0.5",en,no-cache,close,0,48,3,0


In [3]:
pip install pycaret

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from pycaret.clustering import setup, create_model, evaluate_model, assign_model, save_model, plot_model

sample_data = df.sample(frac=0.1, random_state=42)  # random_state for reproducibility

# Initialize the PyCaret clustering environment
clustering_setup = setup(data=sample_data, session_id=123, normalize=True, verbose=False, memory=None)

# Create a specific clustering model, e.g., KMeans
kmeans_model = create_model('kmeans')

# Evaluate the model
evaluate_model(kmeans_model)

# Assign clusters to the data
clustered_data = assign_model(kmeans_model)

# Display the first few rows of the clustered data
print(clustered_data.head())

# Save the model
save_model(kmeans_model, 'clustering_model')

# Optional: Plot clusters
plot_model(kmeans_model, plot='cluster')


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3512,3.4780,0.4280,0,0,0


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

      Method                                           Full URL  \
28553   POST   http://localhost:8080/tienda1/publico/entrar.jsp   
1246     GET   http://localhost:8080/tienda1/miembros/salir.jsp   
59943   POST   http://localhost:8080/tienda1/publico/entrar.jsp   
3698     GET  http://localhost:8080/tienda1/miembros/imagene...   
41836    GET  http://localhost:8080/tienda1/publico/caracter...   

                                                URL Path Query Params  \
28553                        /tienda1/publico/entrar.jsp          NaN   
1246                         /tienda1/miembros/salir.jsp          NaN   
59943                        /tienda1/publico/entrar.jsp          NaN   
3698               /tienda1/miembros/imagenes/zarauz.jpg          NaN   
41836  /tienda1/publico/caracteristicas.jsp/asf-logo-...          NaN   

       Query Params Length  Number of Query Params  Body Params  \
28553                    0                       0          NaN   
1246                    